In [1]:
import rebound
import numpy as np
import multiprocessing as mp
import time as timing

In [2]:
def r_power(slope, min_v, max_v): #powerlaw
    y = np.random.uniform()
    pow_max = pow(max_v, slope+1.)
    pow_min = pow(min_v, slope+1.)
    return pow((pow_max-pow_min)*y + pow_min, 1./(slope+1.))

def r_uni(minimum, maximum): #uniform
    return np.random.uniform()*(maximum-minimum)+minimum

def r_ray(sigma): #rayleigh
    return sigma*np.sqrt(-2*np.log(np.random.uniform()))

In [3]:
def get_a(sim,planet_index):
    particles = sim.particles
    com = sim.get_com
    p = particles[planet_index]
    mu = sim.G*(com.m+p.m)
    dvx = p.vx - com.vx
    dvy = p.vy - com.vy
    dvz = p.vz - com.vz
    v2 = dvx*dvx + dvy*dvy + dvz*dvz
    dx = p.x - com.x
    dy = p.y - com.y
    dz = p.z - com.z
    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return -mu/(v2 - 2.*mu/r)

In [4]:
def get_outputs(sim,E0,start_t):
    dE = fabs((sim.energy() - E0)/E0)
    current_t = timing.time()-start_t
    a = get_a(sim,sim.N_active-1)
    N_mini = 0
    if(sim.integrator == "hybarid"):
        N_mini = sim.ri_hybarid.mini.N
    return sim.t, dE, a, sim.N, N_mini, current_t

In [6]:
def problem(HSR, dt):
    name = "output/Kirsh_HSR"+str(HSR)+"_dt"+str(dt)+".txt"
    
    sim = rebound.Simulation()

    #integrator options
    sim.integrator = "hybarid"
    sim.ri_hybarid.switch_radius = HSR
    sim.ri_hybarid.CE_radius = 20.
    sim.dt = dt
    sim.testparticle_type = 1
    #tmax = 1e5*6.283
    tmax = 10

    #collision and boundary options
    sim.collision = "direct"
    sim.collision_resolve = "merge"
    sim.boundary = "open"
    boxsize = 3.
    sim.configure_box(boxsize)
    sim.collisions_track_dE = 1
    
    #massive bodies - scale earth later
    m_earth = 0.000003003
    a_planet = 25
    m_planet = m_earth*2.3
    r_planet = 0.0000788215 #radius of particle using 2g/cm^3 (AU)
    sim.add(m=1.)
    sim.add(m=m_planet,r=r_planet,a=a_planet,e=0,inc=0.0001)
    
    sim.N_active = sim.N
    
    #planetesimal disk
    m_pl = m_planet/600.
    #N_pl = 230*m_earth/m_pl
    N_pl = 10
    r_pl = 0.00000934532
    disk_min_a = a_planet - 10.5
    disk_max_a = a_planet + 10.5
    while sim.N < (N_pl + sim.N_active):
        a = r_power(1, disk_min_a, disk_max_a)
        e = r_ray(0.01)
        inc = r_ray(0.005)
        sim.add(m=m_pl, r=r_pl, a=a, e=e, inc=inc, Omega=r_uni(0,2*np.pi), omega=r_uni(0,2*np.pi), f=r_uni(0,2*np.pi))
    
    #final physics ini
    sim.move_to_com()
    E0 = sim.calculate_energy()
    
    #writing ini
    write_t = sim.dt
    write_log = 1.01  #log output
    write_lin = 100   #linear output
    
    start_t = timing.time()
    while sim.t < tmax:
        sim.integrate(write_t)
        with open(name, "a") as writefile:
            writefile.write(','.join(map(str, get_outputs(sim,E0,start_t))) )
        write_t = min(write_t*write_log,write_t*write_lin)

In [9]:
dt = 12.76
HSR = 3


array([ 1. ,  3.5,  6. ])